In [2]:
!pip install selenium
!pip install pandas numpy tqdm selenium webdriver_manager beautifulsoup4

In [6]:
import csv
import pandas as pd
import numpy as np
import time
import requests

from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from itertools import repeat
from urllib.request import urlopen
import re

chromedriver = '/home/user/다운로드'
driver = webdriver.Chrome()

# 무신사 페이지 접근
url = "https://www.musinsa.com/app/"
# options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성 -> 속도 향상 위함
# options.add_argument('headless') # headless 모드 설정 -> 웹 화면 안보여줌
# options.add_argument("disable-gpu") # gpu를 사용하지 않는 옵션(에러 발생 시 --disable-gpu 로 변경)
driver.get(url)
time.sleep(1)
driver.maximize_window()

def get_from_page_musinsa_no(b_catagory_no, start_page, page):
        musinsa_link_li = []  # 무신사 제품 no 리스트
        
        # GET page 개수만큼 무신사 페이지에서 무신사 제품 no 
        for p in tqdm(range(start_page, start_page+page)):
                url = f'https://www.musinsa.com/categories/item/{b_catagory_no}?d_cat_cd={b_catagory_no}&brand=&list_kind=small&sort=pop_category&sub_sort=&page={p}&display_cnt=90\
                        &group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='
                driver.get(url)
                time.sleep(0.1)
                link1 = 'https://www.musinsa.com/app/goods/'

                # GET p페이지에서 무신사 제품 no 리스트
                temp_no_li = []
                temp = driver.find_elements(By.XPATH, '//*[@id="searchList"]/*[@class="li_box"]')
                for code in temp:
                        link2 = code.get_attribute('data-no')
                        temp_no_li.append(link1 + link2)
                
                musinsa_link_li += temp_no_li
        
        return musinsa_link_li
    
# top
b_catagory_no = '001'
page = 10

musinsa_top_link_li = get_from_page_musinsa_no(b_catagory_no, 1, page) # b_catagory_no, start_page, page
len(musinsa_top_link_li)

# 무신사 top link list -> csv 저장
import csv

with open('/Users/jeongjuyeon/Desktop/project/musinsa_top_link_li.csv','w') as file :

    write = csv.writer(file)
    write.writerow(musinsa_top_link_li)
    

driver.quit()

100%|███████████████████████████████████████████| 10/10 [01:56<00:00, 11.68s/it]


In [23]:
import pandas as pd 
from tqdm import tqdm
import time
import csv

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

musinsa_top_link = list()
f = open('/Users/jeongjuyeon/Desktop/project/musinsa_top_link_li.csv','r')
rea = csv.reader(f)
for row in rea:
    musinsa_top_link.append(row)
f.close



<function TextIOWrapper.close()>

In [25]:
musinsa_top_link = musinsa_top_link[0]
musinsa_top_link

['https://www.musinsa.com/app/goods/3034895',
 'https://www.musinsa.com/app/goods/2995119',
 'https://www.musinsa.com/app/goods/3574305',
 'https://www.musinsa.com/app/goods/2043056',
 'https://www.musinsa.com/app/goods/3411311',
 'https://www.musinsa.com/app/goods/3164171',
 'https://www.musinsa.com/app/goods/2902788',
 'https://www.musinsa.com/app/goods/3057660',
 'https://www.musinsa.com/app/goods/3465073',
 'https://www.musinsa.com/app/goods/3187524',
 'https://www.musinsa.com/app/goods/3007774',
 'https://www.musinsa.com/app/goods/2246471',
 'https://www.musinsa.com/app/goods/2660728',
 'https://www.musinsa.com/app/goods/2114319',
 'https://www.musinsa.com/app/goods/2911578',
 'https://www.musinsa.com/app/goods/2889435',
 'https://www.musinsa.com/app/goods/2113554',
 'https://www.musinsa.com/app/goods/843276',
 'https://www.musinsa.com/app/goods/2139775',
 'https://www.musinsa.com/app/goods/3561722',
 'https://www.musinsa.com/app/goods/3032889',
 'https://www.musinsa.com/app/goods

In [26]:
# empty list
name = []
brand = []
sex = []
link = []

# top size_info datafrmae
top = {"사이즈" : [],
        "총장": [],
        "어깨너비": [],
        "가슴단": [],
        "소매길이": []
        }

top = pd.DataFrame(top)


# 각 상품 데이터 가져오기
for li in tqdm(musinsa_top_link[:10]):
    print(li)
    # 해당페이지 링크(각 상품별)
    product_link = f"{li}"

    try:
        req = Request(product_link, headers={"user-agent":"Chrome"})
        response = urlopen(req)
        soup = BeautifulSoup(response, "html.parser")
    except:
        continue

    try:
        # size_info
        first_table = soup.find('table', class_='table_th_grey')
        df = pd.read_html(str(first_table))[0]
        df.drop([0], axis=0, inplace=True)
        df.rename(columns={"cm" : "사이즈"}, inplace=True)
        df = df.iloc[:, :-1]
        top = pd.concat([top, df], ignore_index=True)                

        # 제품명
        product_name = soup.select_one("#page_product_detail > div.right_area.page_detail_product > div.right_contents.section_product_summary > span > em").text
        name.extend([product_name] * len(range(2, len(soup.select("#size_table > tbody > tr")[2:])+ 2)))

        #  Product Info(브랜드)
        product_brand = soup.select_one("#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong > a").text
        brand.extend([product_brand] * len(range(2, len(soup.select("#size_table > tbody > tr")[2:])+ 2)))

        #  Product Info(성별)
        try:
            product_sex = soup.select_one("#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(2) > p.product_article_contents > span.txt_gender").text.replace("\n", "")
        except:
            product_sex = soup.select_one("#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(3) > p.product_article_contents > span").text.replace("\n", "")

        sex.extend([product_sex] * len(range(2, len(soup.select("#size_table > tbody > tr")[2:])+ 2)))

        # link
        link.extend([product_link] * len(range(2, len(soup.select("#size_table > tbody > tr")[2:])+ 2)))
    except:
        print('error4')
        continue
    
    # Dataframe
    top["제품명"] = name
    top["브랜드"] = brand
    top["성별"] = sex
    top["링크"] = link

  0%|                                                    | 0/10 [00:00<?, ?it/s]

https://www.musinsa.com/app/goods/3034895


 10%|████▍                                       | 1/10 [00:01<00:09,  1.06s/it]

https://www.musinsa.com/app/goods/2995119


 20%|████████▊                                   | 2/10 [00:01<00:06,  1.32it/s]

...

 80%|███████████████████████████████████▏        | 8/10 [00:06<00:01,  1.16it/s]

https://www.musinsa.com/app/goods/3465073


 90%|███████████████████████████████████████▌    | 9/10 [00:07<00:00,  1.20it/s]

https://www.musinsa.com/app/goods/3187524


100%|███████████████████████████████████████████| 10/10 [00:08<00:00,  1.23it/s]


In [27]:
top

,사이즈,총장,어깨너비,가슴단,소매길이,가슴단면,제품명,브랜드,성별,링크
0,S,63.5,51.5,NaN,58,62,Velour Soccer Jersey Black,SCULPTOR,여,https://www.musinsa.com/app/goods/3034895
1,M,65.5,54,NaN,59,64,Velour Soccer Jersey Black,SCULPTOR,여,https://www.musinsa.com/app/goods/3034895
2,L,68.5,57.5,NaN,62.5,66,Velour Soccer Jersey Black,SCULPTOR,여,https://www.musinsa.com/app/goods/3034895
3,XL,71.5,59.5,NaN,65.5,68,Velour Soccer Jersey Black,SCULPTOR,여,https://www.musinsa.com/app/goods/3034895
4,S,64,65,NaN,52,66,컬러 배색 1992 그래픽 후드 (NAVY),TRILLION,남여,https://www.musinsa.com/app/goods/2995119
5,M,68,56,NaN,61,66,컬러 배색 1992 그래픽 후드 (NAVY),TRILLION,남여,https://www.musinsa.com/app/goods/2995119
6,L,70,58,NaN,63,68,컬러 배색 1992 그래픽 후드 (NAVY),TRILLION,남여,https://www.musinsa.com/app/goods/2995119
7,XL,72,60,NaN,65,70,컬러 배색 1992 그래픽 후드 (NAVY),TRILLION,남여,https://www.musinsa.com/app/goods/2995119
8,M,61,49,NaN,58,53,스탠다드 스웨츠 셔츠,REBRENNER,남,https://www.musinsa.com/app/goods/3574305
9,L,68,52,NaN,60,58,스탠다드 스웨츠 셔츠,REBRENNER,남,https://www.musinsa.com/app/goods/3574305


In [ ]:
!pip install --upgrade webdriver_manager

In [21]:
import csv
import pandas as pd
import numpy as np
import time

from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

data = []
with open('/Users/jeongjuyeon/Desktop/project/musinsa_top_link_li.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)
data = data[0]
len(data)

def get_review(driver, p_name, link, cnt, review_list):
    global on_page
    time.sleep(0.1)

    # 출력 용도
    try:
        on_page = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.paging-btn.btn.active').text
    except:
        pass

    # 한 페이지의 전체 리뷰 리스트(최대 10개씩)
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    review10 = soup.find_all(class_='review-list')

    for i in range(len(review10)):
        if cnt == 10:
            break

        try:
            # 프로필 정보
            profile_info = review10[i].find(class_='review-profile__text-wrap')
            profile_name = profile_info.find(class_='review-profile__name').text.strip()
            profile_date = profile_info.find(class_='review-profile__date').text.strip()
            body_info = profile_info.find(class_='review-profile__body_information').text.strip()

            # 상품 정보
            goods_info = review10[i].find(class_='review-goods-information__item')
            goods_name = goods_info.find(class_='review-goods-information__name').text.strip()
            goods_option = goods_info.find(class_='review-goods-information__option').text.strip()

            # 별점
            rating = review10[i].find(class_='review-list__rating__unit').text.strip()

            # 평가
            evaluation_items = review10[i].find_all(class_='review-evaluation--type3__item')
            size_evaluation = '평가 없음'
            brightness_evaluation = '평가 없음'
            color_evaluation = '평가 없음'
            thickness_evaluation = '평가 없음'
            for item in evaluation_items:
                if '사이즈' in item.text:
                    size_evaluation = item.span.text.strip()
                elif '밝기' in item.text:
                    brightness_evaluation = item.span.text.strip()
                elif '색감' in item.text:
                    color_evaluation = item.span.text.strip()
                elif '두께감' in item.text:
                    thickness_evaluation = item.span.text.strip()

            # 리뷰 내용
            review_content = review10[i].find(class_='review-contents__text').text.strip()

            # 추가 이미지
            image_elements = review10[i].find_all(class_='review-content-photo__item')
            images = [image['style'].split('background-image:url(')[1].split(');')[0] for image in image_elements]

            #print(f"Adding review: {p_name}, {goods_name}, {size_evaluation}, {brightness_evaluation}, {color_evaluation}, {thickness_evaluation}")

            review_list.append({
                '제품명': p_name,
                '리뷰어': profile_name,
                '날짜': profile_date,
                '신체정보': body_info,
                '상품명': goods_name,
                '옵션': goods_option,
                '별점': rating,
                '사이즈평가': size_evaluation,
                '밝기평가': brightness_evaluation,
                '색감평가': color_evaluation,
                '두께감평가': thickness_evaluation,
                '리뷰내용': review_content,
                '이미지': images,
                '링크': link
            })
            cnt += 1
        except Exception as e:
            print(f"An error occurred while adding review: {e}")
            continue

    return on_page, cnt, review_list

options = webdriver.ChromeOptions()
options.add_argument("headless")

review_list = []

# 0~1799(1800개)번째 리뷰 추출
for link in tqdm(data[0:100]):
    try:
        driver = webdriver.Chrome()
        driver.get(link)
        
        # 추천 순 클릭
        driver.find_element(By.ID, 'reviewSelectSort').send_keys('추천 순')
        time.sleep(0.1)

        # 페이지 크롤링
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')

        # 제품명
        p_name = soup.find(class_='product_title').em.text.strip()

        # 리뷰 페이지 전체 수
        try:
            total_page = soup.find('div','box_page_msg').text.strip().split(' 페이지')[0]
        except:
            driver.close()
            continue
        
        # 제품 당 10개 리뷰 가져오기 위한 카운트
        cnt = 0

        # 첫 페이지 크롤링
        on_page, cnt, review_list = get_review(driver, p_name, link, cnt, review_list)
        
        # 페이지 넘기기
        flag=True
        while flag:
            for i in range(4, 9):
                
                if (cnt== 10) or (on_page == total_page):
                    flag=False
                    break

                page_ = f'#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child({i})'
                
                try:
                    driver.find_element(By.CSS_SELECTOR, page_).send_keys(Keys.ENTER)
                except:
                    page_ = f'#reviewListFragment > div:nth-child(3) > div.nslist_bottom > div.pagination.textRight > div > a:nth-child({i})'
                    driver.find_element(By.CSS_SELECTOR, page_).send_keys(Keys.ENTER)
                    
                
                on_page, cnt, review_list = get_review(driver, p_name, link, cnt, review_list)

        driver.close()
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

df = pd.DataFrame(review_list)
print(df)

  2%|▊                                        | 2/100 [02:12<1:47:39, 65.92s/it]

An error occurred: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.71)
Stacktrace:
0   chromedriver                        0x0000000104b844cc chromedriver + 4162764
1   chromedriver                        0x0000000104b7c654 chromedriver + 4130388
2   chromedriver                        0x00000001047d3bc0 chromedriver + 293824
3   chromedriver                        0x00000001047bb0b0 chromedriver + 192688
4   chromedriver                        0x00000001047bafd8 chromedriver + 192472
5   chromedriver                        0x00000001048592b0 chromedriver + 840368
6   chromedriver                        0x000000010480d6bc chromedriver + 530108
7   chromedriver                        0x000000010480e930 chromedriver + 534832
8   chromedriver                        0x0000000104b49df8 chromedriver + 3923448
9   chromedriver                        0x0000000104b4e3cc chromedr

  3%|█▏                                      | 3/100 [05:19<3:15:27, 120.91s/it]

An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: list index out of range


  4%|█▌                                      | 4/100 [06:27<2:40:09, 100.10s/it]

...

An error occurred while adding review: list index out of range


 99%|███████████████████████████████████████▌| 99/100 [1:46:00<01:22, 82.41s/it]

An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'
An error occurred while adding review: 'NoneType' object has no attribute 'text'


100%|███████████████████████████████████████| 100/100 [1:47:10<00:00, 64.31s/it]

                            제품명            리뷰어          날짜  \
0    Velour Soccer Jersey Black  LV.5 dayyeonn  2023.02.07   
1    Velour Soccer Jersey Black       LV.6 현세현  2023.03.10   
2    Velour Soccer Jersey Black      LV.7 은공주!  2023.04.18   
3    Velour Soccer Jersey Black   LV.4 서진.....  2023.02.10   
4    Velour Soccer Jersey Black     LV.6 on.S.  2023.02.18   
..                          ...            ...         ...   
755        Longsleeve 무지 긴팔티 블랙   LV.7 이집셔츠잘하네  2021.06.08   
756        Longsleeve 무지 긴팔티 블랙  LV.5 무 신사 망해라  2023.05.27   
757        Longsleeve 무지 긴팔티 블랙       LV.5 리쓰귯  2023.04.11   
758        Longsleeve 무지 긴팔티 블랙       LV.4 돈줘봐  2021.09.04   
759        Longsleeve 무지 긴팔티 블랙   LV.7 댕댕이보더콜리  2021.02.25   

                   신체정보                         상품명  옵션 별점  사이즈평가   밝기평가  \
0     여성 · 164cm · 54kg  Velour Soccer Jersey Black   M     보통이에요  보통이에요   
1     남성 · 176cm · 80kg  Velour Soccer Jersey Black  XL     보통이에요  보통이에요   
2     여성 · 168cm · 53kg  Ve

In [23]:
df.to_csv('/Users/jeongjuyeon/Desktop/project/top_review.csv', encoding='utf-8')

In [25]:
!pip install koreanize-matplotlib

import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from matplotlib import font_manager, rc
import koreanize_matplotlib
%matplotlib inline


     |████████████████████████████████| 7.9 MB 517 kB/s eta 0:00:01


In [27]:
df = pd.read_csv('/Users/jeongjuyeon/Desktop/project/top_review.csv', encoding='utf-8')
df

,Unnamed: 0,제품명,리뷰어,날짜,신체정보,상품명,옵션,별점,사이즈평가,밝기평가,색감평가,두께감평가,리뷰내용,이미지,링크
0,0,Velour Soccer Jersey Black,LV.5 dayyeonn,2023.02.07,여성 · 164cm · 54kg,Velour Soccer Jersey Black,M,NaN,보통이에요,보통이에요,보통이에요,보통이에요,너무 예뻐요 스컬....역시 믿고사는 스컬프터 흑흑 다들 고민하지 말고 사십쇼,"[""'//image.msscdn.net/data/estimate/3034895_0/...",https://www.musinsa.com/app/goods/3034895
1,1,Velour Soccer Jersey Black,LV.6 현세현,2023.03.10,남성 · 176cm · 80kg,Velour Soccer Jersey Black,XL,NaN,보통이에요,보통이에요,보통이에요,보통이에요,살까말까 고민을 많이 했는데 너무 이뻐요 소재도 핏도 디자인도 다 맘에 들어요,"[""'//image.msscdn.net/data/estimate/3034895_0/...",https://www.musinsa.com/app/goods/3034895
2,2,Velour Soccer Jersey Black,LV.7 은공주!,2023.04.18,여성 · 168cm · 53kg,Velour Soccer Jersey Black,M,NaN,보통이에요,보통이에요,보통이에요,보통이에요,부드럽고 지금계절에 입기 딱 좋아요! 추천합니당,"[""'//image.msscdn.net/data/estimate/3034895_0/...",https://www.musinsa.com/app/goods/3034895
3,3,Velour Soccer Jersey Black,LV.4 서진.....,2023.02.10,여성 · 155cm · 150kg,Velour Soccer Jersey Black,L,NaN,커요,어두워요,선명해요,보통이에요,흐물흐물거려서 너무 예뻐요 ! 부디 밖에 나갔다가 겹치는 일만 없기를..근데 키 크...,"[""'//image.msscdn.net/data/estimate/3034895_0/...",https://www.musinsa.com/app/goods/3034895
4,4,Velour Soccer Jersey Black,LV.6 on.S.,2023.02.18,남성 · 165cm · 52kg,Velour Soccer Jersey Black,M,NaN,보통이에요,보통이에요,선명해요,보통이에요,진짜 너무 예쁜거 아닙니까 핏 미치도록 예뻐요 남자 분들도 입어도 너무 예쁜 옷이에요,"[""'//image.msscdn.net/data/estimate/3034895_0/...",https://www.musinsa.com/app/goods/3034895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,755,Longsleeve 무지 긴팔티 블랙,LV.7 이집셔츠잘하네,2021.06.08,남성 · 174cm · 64kg,Longsleeve 무지 긴팔티 블랙,L,NaN,보통이에요,어두워요,선명해요,보통이에요,벌써 3번째 재구매입니다.다른곳에서도 쭉티사봣는데 역시 꼼파뇨 쭉티가 퀄리티는 갑이...,"[""'//image.msscdn.net/data/estimate/394608_0/g...",https://www.musinsa.com/app/goods/394608
756,756,Longsleeve 무지 긴팔티 블랙,LV.5 무 신사 망해라,2023.05.27,남성 · 185cm · 75kg,Longsleeve 무지 긴팔티 블랙,L,NaN,커요,보통이에요,보통이에요,보통이에요,데일리로 입기좋은 롱슬리브.약간 두깨가 있는편. 팔다이 긴사람들 입어도넉넉함.,"[""'//image.msscdn.net/data/estimate/394608_0/g...",https://www.musinsa.com/app/goods/394608
757,757,Longsleeve 무지 긴팔티 블랙,LV.5 리쓰귯,2023.04.11,남성 · 175cm · 82kg,Longsleeve 무지 긴팔티 블랙,M,NaN,보통이에요,보통이에요,보통이에요,보통이에요,두께 적당하고 딱 기본 긴팔티에요 질도 괜찮네요이쁩니다,"[""'//image.msscdn.net/data/estimate/394608_0/g...",https://www.musinsa.com/app/goods/394608
758,758,Longsleeve 무지 긴팔티 블랙,LV.4 돈줘봐,2021.09.04,남성 · 177cm · 76kg,Longsleeve 무지 긴팔티 블랙,M,NaN,보통이에요,보통이에요,보통이에요,보통이에요,예뻐요 진짜 지금 계절에 입기에 딱 좋아요 추천해용,"[""'//image.msscdn.net/data/estimate/394608_0/g...",https://www.musinsa.com/app/goods/394608


In [40]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('/Users/jeongjuyeon/Desktop/project/top_review.csv', encoding='utf-8')

# '신체 정보' 열을 기반으로 '성별', '키', '몸무게' 열 생성
df[['성별', '키', '몸무게']] = df['신체정보'].str.extract(r'(여성|남성) · (\d+)cm · (\d+)kg')

# 결과 확인
print(df.head())

# 결과를 새로운 CSV 파일로 저장
df.to_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified.csv', encoding='utf-8', index=False)


   Unnamed: 0                         제품명            리뷰어          날짜  \
0           0  Velour Soccer Jersey Black  LV.5 dayyeonn  2023.02.07   
1           1  Velour Soccer Jersey Black       LV.6 현세현  2023.03.10   
2           2  Velour Soccer Jersey Black      LV.7 은공주!  2023.04.18   
3           3  Velour Soccer Jersey Black   LV.4 서진.....  2023.02.10   
4           4  Velour Soccer Jersey Black     LV.6 on.S.  2023.02.18   

                 신체정보                         상품명  옵션  별점  사이즈평가   밝기평가  \
0   여성 · 164cm · 54kg  Velour Soccer Jersey Black   M NaN  보통이에요  보통이에요   
1   남성 · 176cm · 80kg  Velour Soccer Jersey Black  XL NaN  보통이에요  보통이에요   
2   여성 · 168cm · 53kg  Velour Soccer Jersey Black   M NaN  보통이에요  보통이에요   
3  여성 · 155cm · 150kg  Velour Soccer Jersey Black   L NaN     커요   어두워요   
4   남성 · 165cm · 52kg  Velour Soccer Jersey Black   M NaN  보통이에요  보통이에요   

    색감평가  두께감평가                                               리뷰내용  \
0  보통이에요  보통이에요        너무 예뻐요 스컬....역시 믿고사는 스컬

In [52]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified.csv', encoding='utf-8')

# 제품명에서 특정 키워드를 찾아내어 카테고리로 사용
def extract_category(product_name):
    keywords = ['셔츠', '스웨츠 셔츠', '트레이닝', '맨투맨', '후드', '져지', '집업', '긴팔', '반팔', '니트', '카라', '반팔' ]  # 예시 키워드, 필요에 따라 추가 및 수정
    for keyword in keywords:
        if keyword in product_name:
            return keyword
    return '기타'  # 키워드가 없을 경우 기타로 지정

# '카테고리' 열 추가
df['카테고리'] = df['제품명'].apply(extract_category)

# 결과 확인
print(df[['제품명', '카테고리']].head())

# 결과를 새로운 CSV 파일로 저장
df.to_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified2.csv', encoding='utf-8', index=False)


                          제품명 카테고리
0  Velour Soccer Jersey Black   기타
1  Velour Soccer Jersey Black   기타
2  Velour Soccer Jersey Black   기타
3  Velour Soccer Jersey Black   기타
4  Velour Soccer Jersey Black   기타


In [78]:
import pandas as pd

# 파일 경로 설정
file_path = '/Users/jeongjuyeon/Desktop/project/musinsatop_review.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 사용자로부터 카테고리 입력 받기
user_category = input("원하는 카테고리를 입력하세요: ")

# 입력 받은 카테고리에 해당하는 행만 선택
filtered_rows = df[df['카테고리'] == user_category]

# 선택한 열만 추출
selected_columns = ['제품명', '리뷰내용', '링크']
selected_df = filtered_rows[selected_columns]

# 결과 출력
print(selected_df)



원하는 카테고리를 입력하세요: 후드
                     제품명                                               리뷰내용  \
30               빅스타 후드티  진짜 보자마자 팔짝 뛰면서 구매했는데 너무나도 잘 산거 같아요. 재질은 부드러워서 ...   
31               빅스타 후드티  기모인 줄 모르고 샀는데 그래도 너무너무 이뻐서 좋아요 사이즈는 제가 키가 작아서 ...   
32               빅스타 후드티  핏이 너무 이쁘고 데일리로 착용하기 딱 좋은 제품이에요. 팔이 좀 길게 나왔고 저한...   
33               빅스타 후드티  165~167/8 분들.. 오버핏광인이라고 냅다 2사는건 한번만 다시 생각해보세요 ...   
34               빅스타 후드티  이뻐용⚡️⚡️⚡️😮‍💨😮‍💨🤣😮‍💨🌪️😮‍💨🌪️😮‍💨🌀😮‍💨🤗😒리히리리레렐래라라래ㅗ...   
..                   ...                                                ...   
745  월계수 자수 크롭 후드 - 6COL        무난무난 하면서 핏이 귀여운 옷입니당제가 딱 원하는 크롭 길이감이라 만족해요🌿   
746  월계수 자수 크롭 후드 - 6COL  진짜 사는거 추천드려요.너무 이뻐요 진짜 딱 크롭의 정석이랄까.. 손 들면 배보이는...   
747  월계수 자수 크롭 후드 - 6COL  진짜 얇고 기장도 엄청 짧아요. 어중간한 기장이 아니라 더 예쁘긴 해요. 모자 크기...   
748  월계수 자수 크롭 후드 - 6COL      애매한 크롭 기장이 아니고 확실히 짧은 기장이라 좋네요! 핏도 예쁘고 맘에 듭니다   
749  월계수 자수 크롭 후드 - 6COL                 길이감도 적당하고 따뜻하고 이쁩니다! 가격 저렴하게 잘샀어요!   

                               

In [86]:
import pandas as pd

# 파일 경로 설정
file_path = '/Users/jeongjuyeon/Desktop/project/musinsatop_review.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

try:
    # 사용자로부터 카테고리, 성별, 키, 몸무게 입력 받기
    user_category = input("원하는 카테고리를 입력하세요: ")
    user_gender = input("성별을 입력하세요 (남성 또는 여성): ")
    user_height = float(input("키를 입력하세요: "))
    user_weight = float(input("몸무게를 입력하세요: "))

    # 필터링 조건 설정
    category_condition = (df['카테고리'] == user_category) if user_category else True
    gender_condition = (df['성별'] == user_gender) if user_gender else True
    height_condition = (df['키'] >= user_height - 5) & (df['키'] <= user_height + 5) if user_height else True
    weight_condition = (df['몸무게'] >= user_weight - 5) & (df['몸무게'] <= user_weight + 5) if user_weight else True

    # 조건에 맞는 행 출력
    result_df = df[category_condition & gender_condition & height_condition & weight_condition]

    # 결과 출력 또는 "데이터가 부족합니다" 출력
    if not result_df.empty:
        print(result_df[['제품명', '리뷰내용', '링크', '키', '몸무게', '성별']])
    else:
        print("데이터가 부족합니다.")

except ValueError:
    print("입력값의 형식이 잘못되었습니다.")


원하는 카테고리를 입력하세요: 맨투맨
성별을 입력하세요 (남성 또는 여성): 남성
키를 입력하세요: 175
몸무게를 입력하세요: 75
                             제품명  \
52          SPACE CAT 특양면 맨투맨_B1   
399   [기모버전추가]시그니처 오버핏 맨투맨(4col)   
407   [기모버전추가]시그니처 오버핏 맨투맨(4col)   
436  ARCH 1989 맨투맨 (STMSTD-0008)   
439  ARCH 1989 맨투맨 (STMSTD-0008)   
441  ARCH 1989 맨투맨 (STMSTD-0008)   
443  ARCH 1989 맨투맨 (STMSTD-0008)   
596         CGP 아치 로고 맨투맨_멜란지그레이   
598         CGP 아치 로고 맨투맨_멜란지그레이   
678            VNBC 버스터즈 특양면 맨투맨   
684    [22FW] 빅 트위치 루즈핏 맨투맨 NAVY   
685    [22FW] 빅 트위치 루즈핏 맨투맨 NAVY   

                                                  리뷰내용  \
52   배송이 조금 아쉬웠지만 명절 연휴기간동안 물량이 많아서 늦어졌다고 빠른 답변을 받았...   
399               생각했던 것보다 더 이쁘고 색도 바지랑 잘 어울려서 너무 좋아요!   
407  핏이 진짜 잘빠져서 아무거나 걸쳐도 이쁘구요 소재도 탄탄하고 초록색 색감도 정말 이...   
436  옷 짱짱한게 좋네요사이즈도 잘 맞고 퀄이 떨어지거나 라진 않아요기모도 따뜻하고 만족...   
439                  옷또 따시고 사이즈도 딱맞고 코디하기 좋을것같아요 만족합니다   
441    1989스탠다드 처음 구매해보는데사이즈도 딱좋고옷도 이뻐요기본맨투맨으로 딱입니다😀😀😀   
443  기본 회색 라운드에 파란색숫자가 있어서 너무 지루하지않아요~ 안에 기모도 좋구요 

In [58]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

import pandas as pd

# 데이터프레임 읽기
df = pd.read_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified2.csv', encoding='utf-8')

# 데이터프레임의 열을 순회하면서 데이터 유형 확인
for column in df.columns:
    if df[column].dtype == 'float64':
        # float64 형식인 경우, 해당 열의 모든 값을 문자열로 변환
        df[column] = df[column].astype(str)

# 변경된 데이터프레임을 다시 CSV 파일로 저장
df.to_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified2.csv', encoding='utf-8', index=False)


# CSV 파일 불러오기
df = pd.read_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified2.csv', encoding='utf-8')

# 필요한 열만 선택
selected_columns = ['제품명', '카테고리', '옵션', '성별', '키', '몸무게']
df_selected = df[selected_columns]

# 결측치 제거
df_selected = df_selected.dropna()

# 옵션 열을 원핫인코딩
df_onehot = pd.get_dummies(df_selected, columns=['옵션'])

# 카테고리 열을 Tfidf 벡터화
tfidf_vectorizer = TfidfVectorizer()
category_matrix = tfidf_vectorizer.fit_transform(df_onehot['카테고리'])

# 성별, 키, 몸무게 열을 정규화하여 추가
df_normalized = df_onehot.copy()
df_normalized['키'] = df_normalized['키'] / df_normalized['키'].max()
df_normalized['몸무게'] = df_normalized['몸무게'] / df_normalized['몸무게'].max()

# 성별, 키, 몸무게 열을 문자열로 변환
df_normalized['성별'] = df_normalized['성별'].astype(str)
df_normalized['키'] = df_normalized['키'].astype(str)
df_normalized['몸무게'] = df_normalized['몸무게'].astype(str)

# 텍스트와 정규화된 숫자 데이터를 합침
feature_matrix = hstack([category_matrix, df_normalized[['성별', '키', '몸무게']].values])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)

# 제품 추천 함수 정의
def recommend_product(product_name, cosine_sim=cosine_sim):
    idx = df_selected[df_selected['제품명'] == product_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # 제품 자신을 제외한 상위 5개의 유사한 제품
    product_indices = [i[0] for i in sim_scores]
    return df_selected.iloc[product_indices]

# 예시: 'Velour Soccer Jersey Black' 제품과 유사한 제품 추천
recommended_products = recommend_product('Velour Soccer Jersey Black')
print(recommended_products)


TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

In [49]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# CSV 파일 불러오기
df = pd.read_csv('/Users/jeongjuyeon/Desktop/project/top_review_modified.csv', encoding='utf-8')

# 필요한 열만 선택
user_features = df[['성별', '키', '몸무게']]
item_features = df[['상품명', '사이즈평가', '밝기평가', '색감평가', '두께감평가']]

# 성별을 이진수로 변환 (여성: 0, 남성: 1)
user_features['성별'] = (user_features['성별'] == '남성').astype(int)

# '키'와 '몸무게'를 숫자로 변환
user_features['키'] = pd.to_numeric(user_features['키'], errors='coerce')
user_features['몸무게'] = pd.to_numeric(user_features['몸무게'], errors='coerce')

# 결측치를 평균값으로 대체
user_features = user_features.fillna(user_features.mean())

# 데이터 표준화 (Standardization)
scaler = StandardScaler()
user_features_scaled = scaler.fit_transform(user_features[['성별', '키', '몸무게']])

# 특성 벡터 결합
features_combined = pd.concat([pd.DataFrame(user_features_scaled, columns=['성별', '키', '몸무게']),
                               item_features.drop('상품명', axis=1)],
                              axis=1)

# 코사인 유사도 계산
cosine_sim = cosine_similarity(features_combined)

# 추천 함수 정의
def recommend_products(user_index, top_n=5):
    # 유저가 선택한 아이템의 인덱스 추출
    selected_item_index = df.index[df['상품명'] == df.loc[user_index, '상품명']].tolist()[0]

    # 해당 아이템과 유사한 아이템 추출
    sim_scores = list(enumerate(cosine_sim[selected_item_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    # 추천 아이템의 인덱스 및 유사도 반환
    recommended_indices = [i[0] for i in sim_scores]
    return df.iloc[recommended_indices][['상품명', '사이즈평가', '밝기평가', '색감평가', '두께감평가']]

# 특정 유저에 대한 추천 실행 (예: 첫 번째 유저)
user_index = 0
recommended_products = recommend_products(user_index)
print(recommended_products)


                                      상품명  사이즈평가   밝기평가   색감평가  두께감평가
414               데일리 라운드넥 피셔맨 니트 (IVORY)  보통이에요  보통이에요  보통이에요  보통이에요
694                  스튜디오 아치 로고 후드_라이트그레이  보통이에요  보통이에요  보통이에요  보통이에요
659           레이어드 크루 넥 긴팔 티셔츠_긴 기장 [화이트]  보통이에요  보통이에요  보통이에요  보통이에요
82   우먼즈 크롭 베어프렌즈 엠보자수 후드 오트밀(FCB3TS359W)  보통이에요  보통이에요  보통이에요  보통이에요
83   우먼즈 크롭 베어프렌즈 엠보자수 후드 오트밀(FCB3TS359W)  보통이에요  보통이에요  보통이에요  보통이에요
